<a href="https://colab.research.google.com/github/Dellayfd/DSA4213_Fine-tune_BRRT/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -U transformers peft optuna

In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 16.7 MB/s eta 0:00:00


# Preprocess Dataset

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import optuna
from torch import nn
from torch.nn import CrossEntropyLoss
from sklearn.utils.class_weight import compute_class_weight
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import EarlyStoppingCallback
import wandb
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [7]:
import os
def set_seed(seed=42):
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you use multi-GPU

    # For deterministic behavior in cuDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    optuna.seed = seed

set_seed(42)

In [8]:
splits = {'train': 'data/1123/toxic-chat_annotation_train.csv', 'test': 'data/1123/toxic-chat_annotation_test.csv'}
train= pd.read_csv("hf://datasets/lmsys/toxic-chat/" + splits["train"])
test= pd.read_csv("hf://datasets/lmsys/toxic-chat/" + splits["test"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
df=pd.concat([train,test]).reset_index(drop=True)

In [10]:
train=df.sample(frac=0.8, random_state=42)
validation=df.drop(train.index).sample(frac=0.5, random_state=42)
test=df.drop(validation.index).drop(train.index)

train=train.reset_index(drop=True)
validation=validation.reset_index(drop=True)
test=test.reset_index(drop=True)

# Import Pre-trained BERT

In [12]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
class ClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Define dataset
max_len = 128
train_dataset = ClassificationDataset(train['user_input'], train['toxicity'], tokenizer, max_len)
val_dataset = ClassificationDataset(validation['user_input'], validation['toxicity'], tokenizer, max_len)
test_dataset = ClassificationDataset(test['user_input'], test['toxicity'], tokenizer, max_len)

In [17]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [18]:
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted', zero_division=0)
recall = recall_score(true_labels, predictions, average='weighted', zero_division=0)
f1 = f1_score(true_labels, predictions, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(true_labels, predictions)

print("Model Performance on Test Set Before Training:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Model Performance on Test Set Before Training:
Accuracy: 0.1396
Precision: 0.8423
Recall: 0.1396
F1 Score: 0.1453
Confusion Matrix:
[[ 75 867]
 [  8  67]]


## Balance classes

In [ ]:
# lables in each class

In [19]:
labels = train["toxicity"].values
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print("Class weights:", class_weights)

Class weights: tensor([0.5396, 6.8107])


In [20]:
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(device))
    loss = loss_fct(logits.view(-1, 2), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

In [21]:
class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # <— add **kwargs
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Weighted cross-entropy loss
        if self.class_weights is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(model.device))
        else:
            loss_fct = nn.CrossEntropyLoss()

        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


In [22]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [23]:
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,        # Stop if F1 hasn't improved for 2 evaluation rounds
    early_stopping_threshold=0.01      # Optional: minimum improvement required (0 = any improvement)
)


# LoRA

In [26]:
def model_init_lora(trial):
    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["query", "value"],
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_CLS",
    )
    model = get_peft_model(model, lora_config)
    return model


In [27]:
def lora_objective(trial):
    wandb.init(
        project="bert-lora-optuna",
        name=f"trial_{trial.number}",
        reinit=True
    )

    # Sample hyperparameters
    lora_r = trial.suggest_categorical("lora_r", [8, 16])
    lora_alpha = trial.suggest_categorical("lora_alpha", [32, 64])
    lora_dropout = trial.suggest_categorical("lora_dropout",[0.0, 0.1])
    learning_rate = trial.suggest_categorical("learning_rate", [1e-5,3e-5, 5e-5] )
    warmup_ratio = trial.suggest_categorical("warmup_ratio", [0.0, 0.1])
    batch_size = trial.suggest_categorical("batch_size", [16, 32])

    # Reinitialize base model every trial
    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)

    # Apply LoRA config
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=["query", "value"],
        task_type="SEQ_CLS"
    )
    model = get_peft_model(model, lora_config)

    # Build training args
    training_args = TrainingArguments(
        output_dir=f"./results/trial_{trial.number}",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=8,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        warmup_ratio=warmup_ratio,
        logging_dir=f"./logs/trial_{trial.number}",
        report_to=["wandb"],
        run_name=f"trial_{trial.number}",
        logging_steps=50,
        save_total_limit=2,
        seed=trial.number
    )

    trainer = WeightedTrainer(
        model_init=lambda trial: model_init_lora(trial),
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping],
        class_weights=class_weights
    )

    trainer.train()
    metrics = trainer.evaluate()
    return metrics["eval_f1"]


In [28]:
lora_study = optuna.create_study(direction="maximize")
lora_study.optimize(lora_objective, n_trials=8)

print("Best trial:")
print(lora_study.best_trial.params)

[I 2025-10-17 08:44:55,633] A new study created in memory with name: no-name-d73ed7b5-96c3-41d1-b873-93c493f38444


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2755200925.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'cl

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.527500,0.467153,0.886811,0.371585,0.272000,0.586207
2,0.410100,0.344169,0.910433,0.480000,0.358974,0.724138
3,0.360800,0.285112,0.911417,0.516129,0.375000,0.827586
4,0.417200,0.251857,0.931102,0.573171,0.443396,0.810345
5,0.385600,0.242721,0.949803,0.643357,0.541176,0.793103
6,0.406300,0.230174,0.935039,0.592593,0.461538,0.827586
7,0.320300,0.225824,0.935039,0.592593,0.461538,0.827586


[I 2025-10-17 08:59:31,219] Trial 0 finished with value: 0.6433566433566433 and parameters: {'lora_r': 16, 'lora_alpha': 64, 'lora_dropout': 0.1, 'learning_rate': 5e-05, 'warmup_ratio': 0.0, 'batch_size': 32}. Best is trial 0 with value: 0.6433566433566433.


eval/accuracy,▁▄▄▆█▆▆█
eval/f1,▁▄▅▆█▇▇█
eval/loss,█▄▃▂▁▁▁▁
eval/precision,▁▃▄▅█▆▆█
eval/recall,▁▅█▇▇██▇
eval/runtime,▃▂▂▁▂█▁▂
eval/samples_per_second,▆▇▇█▇▁█▇
eval/steps_per_second,▆▇▇█▇▁█▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
+3,...


...

eval/accuracy,▁▄▆▇█▇███
eval/f1,▁▃▅▆▇▇███
eval/loss,█▄▃▃▃▁▂▂▂
eval/precision,▁▃▅▆▇▆███
eval/recall,▁▃▃▃▁█▆▅▆
eval/runtime,█▆▄▆▆▁▇▇▅
eval/samples_per_second,▁▃▅▃▃█▂▂▄
eval/steps_per_second,▁▃▅▃▃█▂▂▄
train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
+3,...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2755200925.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'cl

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.650300,0.640189,0.647638,0.171296,0.098930,0.637931
2,0.513600,0.463501,0.814961,0.303704,0.193396,0.706897
3,0.396300,0.337392,0.897638,0.474747,0.335714,0.810345
4,0.372600,0.292122,0.903543,0.510000,0.359155,0.879310
5,0.395100,0.272027,0.902559,0.507463,0.356643,0.879310
6,0.297600,0.252171,0.916339,0.540541,0.393701,0.862069
7,0.356800,0.247860,0.909449,0.525773,0.375000,0.879310
8,0.277300,0.244718,0.914370,0.539683,0.389313,0.879310


[I 2025-10-17 11:00:20,872] Trial 7 finished with value: 0.5405405405405406 and parameters: {'lora_r': 8, 'lora_alpha': 32, 'lora_dropout': 0.1, 'learning_rate': 3e-05, 'warmup_ratio': 0.1, 'batch_size': 32}. Best is trial 6 with value: 0.7049180327868853.


Best trial:
{'lora_r': 16, 'lora_alpha': 32, 'lora_dropout': 0.1, 'learning_rate': 3e-05, 'warmup_ratio': 0.0, 'batch_size': 16}


### model with best-performing parameters

In [32]:
lora_study.best_trial.params

{'lora_r': 16,
 'lora_alpha': 32,
 'lora_dropout': 0.1,
 'learning_rate': 3e-05,
 'warmup_ratio': 0.0,
 'batch_size': 16}

In [33]:
lora_model = model_init_lora(6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
training_args = TrainingArguments(
        #output_dir=f"./results/trial_{trial.number}",
        learning_rate=3e-05,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=6,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        warmup_ratio=0,
        #logging_dir=f"./logs/trial_{trial.number}",
        report_to=["wandb"],
        #run_name=f"trial_{trial.number}",
        logging_steps=50,
        save_total_limit=2,
        seed=42
    )

trainer = WeightedTrainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping],
        class_weights=class_weights
    )

trainer.train()

/tmp/ipython-input-2755200925.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.536200,0.452943,0.883858,0.351648,0.258065,0.551724
2,0.576800,0.433692,0.922244,0.447552,0.376471,0.551724
3,0.625800,0.599513,0.948819,0.458333,0.578947,0.379310
4,0.464400,0.404598,0.943898,0.551181,0.507246,0.603448
5,0.466700,0.416397,0.948819,0.559322,0.550000,0.568966
6,0.535100,0.401082,0.954724,0.603448,0.603448,0.603448
7,0.393700,0.398808,0.954724,0.603448,0.603448,0.603448
8,0.460200,0.370487,0.954724,0.616667,0.596774,0.637931


TrainOutput(global_step=4072, training_loss=0.524549515167716, metrics={'train_runtime': 1096.9943, 'train_samples_per_second': 59.304, 'train_steps_per_second': 3.712, 'total_flos': 4294049738096640.0, 'train_loss': 0.524549515167716, 'epoch': 8.0})

In [35]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.4931780993938446,
 'eval_accuracy': 0.9262536873156342,
 'eval_f1': 0.5341614906832298,
 'eval_precision': 0.5,
 'eval_recall': 0.5733333333333334,
 'eval_runtime': 7.5961,
 'eval_samples_per_second': 133.884,
 'eval_steps_per_second': 8.425,
 'epoch': 8.0}

# Full fine-tuning

In [29]:
def full_model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)

In [30]:
def full_objective(trial):
    wandb.init(
        project="bert-full-optuna",
        name=f"trial_{trial.number}",
        reinit=True
    )

    # Sample hyperparameters
    learning_rate = trial.suggest_categorical("learning_rate", [1e-6,1e-5,3e-5] )
    warmup_ratio = trial.suggest_categorical("warmup_ratio", [0.0, 0.1])
    batch_size = trial.suggest_categorical("batch_size", [16, 32])

    # Reinitialize base model every trial
    #model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)

    # Build training args
    training_args = TrainingArguments(
        output_dir=f"./results/trial_{trial.number}",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=8,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        warmup_ratio=warmup_ratio,
        logging_dir=f"./logs/trial_{trial.number}",
        report_to=["wandb"],
        run_name=f"trial_{trial.number}",
        logging_steps=50,
        save_total_limit=2,
        seed=trial.number
    )

    trainer = WeightedTrainer(
        model_init=full_model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping],
        class_weights=class_weights
    )

    trainer.train()
    metrics = trainer.evaluate()
    return metrics["eval_f1"]


In [31]:
full_study = optuna.create_study(direction="maximize")
full_study.optimize(full_objective, n_trials=4)

print("Best trial:")
print(full_study.best_trial.params)

[I 2025-10-17 11:03:29,668] A new study created in memory with name: no-name-7e3acc5d-15da-4f1a-b490-739f7d60a389


eval/accuracy,▁▅███████
eval/f1,▁▄▇▇▇████
eval/loss,█▅▃▂▁▁▁▁▁
eval/precision,▁▃▇▇▇████
eval/recall,▁▃▆██▇██▇
eval/runtime,▅▂█▃▁▅▄▂▅
eval/samples_per_second,▄▇▁▆█▄▄▇▄
eval/steps_per_second,▄▇▁▆█▄▅▇▄
train/epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
+3,...


/tmp/ipython-input-2755200925.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.416900,0.238278,0.953740,0.680272,0.561798,0.862069
2,0.261700,0.246220,0.972441,0.762712,0.750000,0.775862
3,0.166200,0.237955,0.970472,0.769231,0.694444,0.862069
4,0.133700,0.299136,0.977362,0.806723,0.786885,0.827586
5,0.073700,0.348467,0.980315,0.824561,0.839286,0.810345
6,0.122700,0.389272,0.979331,0.817391,0.824561,0.810345
7,0.023300,0.447210,0.978346,0.807018,0.821429,0.793103


[I 2025-10-17 11:30:42,807] Trial 0 finished with value: 0.8245614035087719 and parameters: {'learning_rate': 1e-05, 'warmup_ratio': 0.0, 'batch_size': 32}. Best is trial 0 with value: 0.8245614035087719.


...

eval/accuracy,▁▅▆▅████
eval/f1,▁▆▇▇████
eval/loss,▄▁▃▂▄▇█▄
eval/precision,▄▂▅▁▆██▆
eval/recall,▁▇▇█▇▆▆▇
eval/runtime,█▅▄▃▃▄▁▂
eval/samples_per_second,▁▄▅▆▆▅█▇
eval/steps_per_second,▁▄▅▆▆▅█▇
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇█████
+3,...


/tmp/ipython-input-2755200925.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.371400,0.217469,0.956693,0.694444,0.581395,0.862069
2,0.286100,0.239428,0.972441,0.758621,0.758621,0.758621
3,0.202800,0.311935,0.975394,0.782609,0.789474,0.775862
4,0.119800,0.261001,0.974409,0.790323,0.742424,0.844828
5,0.134500,0.428391,0.977362,0.785047,0.857143,0.724138


[I 2025-10-17 12:37:38,024] Trial 3 finished with value: 0.7903225806451613 and parameters: {'learning_rate': 1e-05, 'warmup_ratio': 0.0, 'batch_size': 32}. Best is trial 0 with value: 0.8245614035087719.


Best trial:
{'learning_rate': 1e-05, 'warmup_ratio': 0.0, 'batch_size': 32}


### model with best-performing hyperparameters

In [36]:
full_study.best_trial.params

{'learning_rate': 1e-05, 'warmup_ratio': 0.0, 'batch_size': 32}

In [41]:
full_model=AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
full_training_args = TrainingArguments(
        #output_dir=f"./results/trial_{trial.number}",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=6,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        warmup_ratio=0.1,
        #logging_dir=f"./logs/trial_{trial.number}",
        report_to=["wandb"],
        #run_name=f"trial_{trial.number}",
        logging_steps=50,
        save_total_limit=2,
        seed=42
    )
full_trainer = WeightedTrainer(
        model=model,
        args=full_training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping],
        class_weights=class_weights
    )

full_trainer.train()

/tmp/ipython-input-2755200925.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.210700,0.255939,0.967520,0.740157,0.681159,0.810345
2,0.246900,0.292424,0.978346,0.813559,0.800000,0.827586
3,0.083400,0.444560,0.975394,0.774775,0.811321,0.741379
4,0.077200,0.473400,0.975394,0.778761,0.800000,0.758621


In [ ]:
full_trainer.evaluate(test_dataset)